In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords

In [2]:
!pip install vaderSentiment

In [3]:
test_data = pd.read_csv( r"C:\Users\cdumitrascu\OneDrive - ENDAVA\Desktop\ML\test.csv",dtype='object')
train_data = pd.read_csv( r"C:\Users\cdumitrascu\OneDrive - ENDAVA\Desktop\ML\train.csv",dtype='object')

In [4]:
#train_data = pd.read_csv("../input/movie-reviews-classification/train.csv")
#test_data = pd.read_csv("../input/movie-reviews-classification/test.csv")

In [5]:
train_data.head()

,id,title_id,primary_title,original_title,is_adult,genres,start_year,end_year,runtime_minutes,text,polarity
0,0,tt0400837,Snuff killer - La morte in diretta,Snuff killer - La morte in diretta,0.0,"Horror,Thriller",2003,NaN,88.0,A cheap exploitation film about a mothers sear...,0
1,1,tt0036038,In Old Oklahoma,In Old Oklahoma,0.0,"Romance,Western",1943,NaN,102.0,John Wayne & Albert Dekker compete for oil rig...,0
2,2,tt0493430,Jackass Number Two,Jackass Number Two,0.0,"Action,Comedy,Documentary",2006,NaN,92.0,This film tops the previous incarnation by a m...,1
3,3,tt0089603,Mishima: A Life in Four Chapters,Mishima: A Life in Four Chapters,0.0,"Biography,Drama",1985,NaN,120.0,"The only pure life, is one that ends with a si...",1
4,4,tt0427038,Carlito's Way: Rise to Power,Carlito's Way: Rise to Power,0.0,"Action,Crime,Drama",2005,NaN,100.0,"really awful... lead actor did OK... the film,...",0


In [6]:
train = train_data[['text','start_year','is_adult','genres']]
test = test_data[['text','start_year','is_adult','genres']]
train_labels=train_data['polarity']

In [7]:
train.head()

,text,start_year,is_adult,genres
0,A cheap exploitation film about a mothers sear...,2003,0.0,"Horror,Thriller"
1,John Wayne & Albert Dekker compete for oil rig...,1943,0.0,"Romance,Western"
2,This film tops the previous incarnation by a m...,2006,0.0,"Action,Comedy,Documentary"
3,"The only pure life, is one that ends with a si...",1985,0.0,"Biography,Drama"
4,"really awful... lead actor did OK... the film,...",2005,0.0,"Action,Crime,Drama"


In [8]:
test.head()

,text,start_year,is_adult,genres
0,"those people,who told me""this movie is good""-s...",2005,0.0,"Action,Crime,Drama"
1,Nick Millard aka Nick Phillips should have lef...,1987,0.0,"Horror,Thriller"
2,"When I saw this movie at age 6, it was in the ...",1985,0.0,"Action,Adventure,Animation"
3,"Not having heard of this film, it came as a su...",2003,0.0,Thriller
4,"Falsely accused, skirt-chasing chums John Wayn...",1935,0.0,"Action,Adventure,Romance"


In [9]:
from string import punctuation
def filter(train):
    train['modi']=train['text'].str.lower()
    i=0
    for x in train['modi']:
        for y in x:
            if y in punctuation:
                train['modi'][i]=train['modi'][i].replace(y,'')
        train['modi'][i]=train['modi'][i].replace('br','')
        i+=1
    train = pd.concat([train.drop(['modi'], axis=1), train['modi'].apply(pd.Series)], axis=1)
filter(train)
filter(test)

c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

In [10]:
train['text'][0]

"A cheap exploitation film about a mothers search for her daughter who has been kidnapped by people who make snuff porno films. The trail leads the mother all over Europe as she searches for her child and we in the audience struggle to stay asleep.<br /><br />This is one of the countless soft-core sleaze films that are made for people who want the excitement of porno with out the stigma or danger of it showing up on their credit card bill.Personally I'd rather have the stigma since those films tend to be more interesting and honest about what we're seeing. This is suppose to be a sexy thriller but its not. Mostly its people talking about things followed by lots of walking from place to place and lead to lead.Periodically through out the film various people get undressed and everything has more than a touch of S&M to the proceedings. The violence and fetish material is of the sort to provoke laughter rather than horror or even excitement, its all so incredibly fake. Worse there is not e

In [11]:
train['modi'][0]

'a cheap exploitation film about a mothers search for her daughter who has been kidnapped by people who make snuff porno films the trail leads the mother all over europe as she searches for her child and we in the audience struggle to stay asleep  this is one of the countless softcore sleaze films that are made for people who want the excitement of porno with out the stigma or danger of it showing up on their credit card billpersonally id rather have the stigma since those films tend to be more interesting and honest about what were seeing this is suppose to be a sexy thriller but its not mostly its people talking about things followed by lots of walking from place to place and lead to leadperiodically through out the film various people get undressed and everything has more than a touch of sm to the proceedings the violence and fetish material is of the sort to provoke laughter rather than horror or even excitement its all so incredibly fake worse there is not even enough nudity to ke

In [12]:
#print(train['modi'])

In [13]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
def filter_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    porter = PorterStemmer()
    stems = []
    for t in tokens:    
        stems.append(porter.stem(t))
    return stems

In [14]:
train['modi']= train['modi'].apply(lambda x : filter_text(x))
test['modi']= test['modi'].apply(lambda x : filter_text(x))

c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
print(train['modi'][0])

['cheap', 'exploit', 'film', 'mother', 'search', 'daughter', 'kidnap', 'peopl', 'make', 'snuff', 'porno', 'film', 'trail', 'lead', 'mother', 'europ', 'search', 'child', 'audienc', 'struggl', 'stay', 'asleep', 'one', 'countless', 'softcor', 'sleaz', 'film', 'made', 'peopl', 'want', 'excit', 'porno', 'stigma', 'danger', 'show', 'credit', 'card', 'billperson', 'id', 'rather', 'stigma', 'sinc', 'film', 'tend', 'interest', 'honest', 'see', 'suppos', 'sexi', 'thriller', 'mostli', 'peopl', 'talk', 'thing', 'follow', 'lot', 'walk', 'place', 'place', 'lead', 'leadperiod', 'film', 'variou', 'peopl', 'get', 'undress', 'everyth', 'touch', 'sm', 'proceed', 'violenc', 'fetish', 'materi', 'sort', 'provok', 'laughter', 'rather', 'horror', 'even', 'excit', 'incred', 'fake', 'wors', 'even', 'enough', 'nuditi', 'keep', 'interest', 'basic', 'par', 'cours', 'mani', 'film', 'youll', 'forgiv', 'lack', 'detail', 'simpli', 'dull', 'bore', 'film', 'stay', 'end', 'hope', 'someth', 'remot', 'prurient', 'occur', '

In [16]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
train['modi']=train['modi'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))
test['modi']=test['modi'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
print(test.loc[1])

text          Nick Millard aka Nick Phillips should have lef...
start_year                                                 1987
is_adult                                                    0.0
genres                                          Horror,Thriller
modi          nick millard aka nick phillip left wellenough ...
Name: 1, dtype: object


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
def vader(train):
    analyser = SentimentIntensityAnalyzer()
    train['vader']=train['text'].apply(lambda x : analyser.polarity_scores(x))
    train = pd.concat([train.drop(['vader'], axis=1), train['vader'].apply(pd.Series)], axis=1)
    train['lenght'] = train['text'].apply(lambda x : len(x))
    train['words'] = train['text'].apply(lambda x :len(x.split(" ")))
    train['blob']=train['text'].apply(lambda x : TextBlob(x).sentiment)
    train = pd.concat([train.drop(['blob'], axis=1), train['blob'].apply(pd.Series)], axis=1)
    return train

In [19]:
def doc2vec(train):
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train["modi"].apply(lambda x: x.split(" ")))]
    model = Doc2Vec(documents, workers=1,vector_size=4,window=1,min_count = 1,seed=10)
    doc2vec_df = train["modi"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
    doc2vec_df.columns = ["d2v_" + str(x) for x in doc2vec_df.columns]
    train = pd.concat([train, doc2vec_df], axis=1)
    
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train["genres"].apply(lambda x: x.split(" ")))]
    # train a Doc2Vec model with our text data
    model = Doc2Vec(documents, workers=1,window=1,vector_size=4,min_count = 1,seed=10)

    # transform each document into a vector data
    doc2vec_df = train["genres"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
    doc2vec_df.columns = ["group_" + str(x) for x in doc2vec_df.columns]
    train = pd.concat([train, doc2vec_df], axis=1)
    
    return train

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def making_features(train,test):
    train = vader(train)
    test = vader(test)
    train = doc2vec(train)
    test = doc2vec(test)
    
    
    tfidf = TfidfVectorizer(min_df=100)
    
    
    tfidf.fit(train["modi"])
    tfidf.fit(test['modi'])
    
    tfidf_result = tfidf.transform(train["modi"]).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
    tfidf_df.index = train.index
    train = pd.concat([train, tfidf_df], axis=1)
    
    
    tfidf_result = tfidf.transform(test["modi"]).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
    tfidf_df.index = test.index
    test = pd.concat([test, tfidf_df], axis=1)
    
    
    train['start_year']=train['start_year'].astype('int')
    train['is_adult']=train['is_adult'].astype('bool')
    test['start_year']=test['start_year'].astype('int')
    test['is_adult']=test['is_adult'].astype('bool')
    train.fillna(0,inplace=True)
    return (train,test)

In [21]:
x,y = making_features(train,test)

c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:
#for o in x.columns:
    #print(o)

In [23]:
print(y.shape)
print(x.shape)
from sklearn.model_selection import train_test_split
ignore_cols = ['text','modi','genres']
features = [c for c in y.columns if c not in ignore_cols]
X_train, X_test, y_train, y_test = train_test_split(x[features], train_labels, test_size=0.33,random_state=0)

(27000, 3290)
(20000, 3290)


In [24]:
ignore_cols = ['text','modi','genres',]
features = [c for c in x.columns if c not in ignore_cols]
print(X_train.shape)

(13400, 3287)


In [25]:
from keras.layers import Dense
from keras.models import Model
from keras.layers import Input, Dense,Activation
from keras.models import Sequential
from keras.layers import Dropout
import keras

model = Sequential([
    Dense(26, input_shape=(x.shape[1]-3,)),
    Activation('relu'),
    Dense(11),
    Activation('relu'),
    Dense(6),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
])


Using TensorFlow backend.
W1028 11:56:58.141414 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1028 11:56:58.154382 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1028 11:56:58.156416 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 26)                85488     
_________________________________________________________________
activation_1 (Activation)    (None, 26)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 11)                297       
_________________________________________________________________
activation_2 (Activation)    (None, 11)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 72        
_________________________________________________________________
activation_3 (Activation)    (None, 6)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 7         
__________

In [27]:
from keras import optimizers
model.compile(optimizers.adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])

W1028 11:56:58.247136 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1028 11:56:58.252120 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1028 11:56:58.258105 19724 deprecation.py:323] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
model.fit(x[features],train_labels,epochs=100,validation_data=(X_test,y_test), batch_size=100,verbose = 1)

W1028 11:57:01.318235 19724 deprecation_wrapper.py:119] From c:\users\cdumitrascu\.conda\envs\pythoncompvision\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 6600 samples
Epoch 1/100
20000/20000 [==============================] - 33s 2ms/step - loss: 0.7351 - acc: 0.5007 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 2/100
20000/20000 [==============================] - 33s 2ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 3/100
20000/20000 [==============================] - 32s 2ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 4/100
20000/20000 [==============================] - 33s 2ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 5/100
20000/20000 [==============================] - 34s 2ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 6/100
20000/20000 [==============================] - 33s 2ms/step - loss: 0.6932 - acc: 0.4993 - val_loss: 0.6931 - val_acc: 0.5035
Epoch 7/100
20000/20000 [==============================] - 33s 2ms/step - loss: 0.6932 - acc: 0.5020 - val_loss: 0.6931 - v

KeyboardInterrupt: 

In [ ]:
ignore_cols = ['text','modi','genres','group']
features = [c for c in y.columns if c not in ignore_cols]


In [ ]:
print(model.evaluate(X_test,y_test))
print(model.evaluate(x[features],train_labels))

In [ ]:
print(model.evaluate(X_test,y_test))
print(model.evaluate(x[features],train_labels))

In [ ]:
y_pred = model.predict(y[features])

In [ ]:
submission = pd.read_csv(r"C:\Users\cdumitrascu\OneDrive - ENDAVA\Desktop\ML\sampleSubmission.csv")
#submission = pd.read_csv('../input/movie-reviews-classification/sampleSubmission.csv')
submission['polarity'] = y_pred
submission.to_csv('submission.csv', index=False)